In [ ]:
from transformers import EncoderDecoderModel, BertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

# Load the full dataset
dataset = load_dataset('Kaludi/Customer-Support-Responses')

# Split the 'train' split into train and test sets
train_split = dataset['train']
train_test_dataset = train_split.train_test_split(test_size=0.2, seed=42)

# Access the train and test sets
train_dataset = train_test_dataset['train']
test_dataset = train_test_dataset['test']

In [ ]:
# Load the tokenizer for the encoder and decoder
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize the inputs and outputs
def tokenize_function(examples):
    inputs = tokenizer(examples['query'], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        outputs = tokenizer(examples['response'], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = outputs["input_ids"]
    return inputs

In [ ]:
# Apply the tokenize function to the train and test datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

# Load the Encoder-Decoder model
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')

# Set the decoder's config
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset
)

# Fine-tune the model
trainer.train()

In [ ]:
# Evaluate the model on the test set
eval_result = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(f"Test set evaluation result: {eval_result}")

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizerFast

# Load the trained model and tokenizer
model = EncoderDecoderModel.from_pretrained('results/checkpoint-12')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Set the decoder's config
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Function to generate a response
def generate_response(input_text):
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

    # Generate the output
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=4,
        early_stopping=True,
        decoder_start_token_id=model.config.decoder_start_token_id
    )
    print(outputs)
    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Chatting loop
while True:
    user_input = input("Enter a query (or 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    response = generate_response(user_input)
    print(f"Generated Response: {response}")